In [25]:
#connects to mysql and creates database and tables

import config
import mysql.connector
from mysql.connector import errorcode
import requests
import json
#database string
endpoint = config.DatabaseEndpoint
name = config.DatabaseName
password = config.DatabasePassword
port = config.DatabasePort
currentOffset = 50
cnx = mysql.connector.connect(
    host = endpoint,
    user = name,
    passwd = password,
)
cur = cnx.cursor()


#table creation SQL
#TABLE SETUP
TABLES = {}
TABLES['businesses'] ="""
CREATE TABLE businesses (
  id VARCHAR(255) PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  review_count INT,
  rating DECIMAL(10,2),
  price INT, 
  address VARCHAR(255)
)
"""
TABLES['reviews'] ="""
CREATE TABLE reviews (
  id VARCHAR(255) PRIMARY KEY,
  text VARCHAR(5000),
  rating DECIMAL(10, 2),
  time_created DATE,
  user VARCHAR(255)
)
"""

#create database and tables

#create_database(cur, "PizzaPlaces")
#create_tables(TABLES, cur)

#get businness information from yelp

businessKeys = ['id', 'name', 'review_count', 
            'rating', 'price', 'location']


add_business = ("INSERT INTO PizzaPlaces.businesses"
                "(id, name, review_count,"
                "rating, price, address) "
                "VALUES (%s, %s, %s, %s ,%s, %s)")

while currentOffset < 1000 :
    params={'term':'pizza place','location':'Brooklyn NY',
        'limit': 50, 'offset':currentOffset}
    #pull
    data = businessCallGET(params).json()
    #clean
    cleaned_data = cleanData(data, businessKeys)
    #inserts data into the table with the add_business column headers
    insertData(cur, add_business, cleaned_data)
    #increment
    curentOffset += 50


in try block to insert into database
successful


In [20]:
#('sUvoBG8t3eKC7WhJbnro2w', 
#'The Pizza Place', 92, 3.5, 2, '1344 BroadwayHewlett, NY 11557')
cur.execute("""
INSERT INTO PizzaPlaces.businesses
    (id, name, review_count, rating, price, address)
VALUES 
    ('sUvoBG8t3eKC7WhJbnro2w', 'The Pizza Place', 92, 3.5, 2,'1344 BroadwayHewlett, NY 11557')
;
""")
           
       ## cleaned_data[0][2], cleaned_data[0][3], cleaned_data[0][4],
        #cleaned_data[0][5]))           

IntegrityError: 1062 (23000): Duplicate entry 'sUvoBG8t3eKC7WhJbnro2w' for key 'PRIMARY'

In [7]:
#creates database
#create database given cursor and name of database
def create_database(cursor, database_name):
    try: #checks if database_name exists
        cursor.execute("USE {}".format(database_name))
    except mysql.connector.Error as err:
        print("Database {} does not exist.".format(database_name))
        #if it doesn't exist, create a new database
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            try:
                cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
                print("Database {} created successfully.".format(database_name))
                #sets database name to new database
                cnx.database = database_name
            except mysql.connector.Error as err:
                print('Failed creating database: {}'.format(err))
                exit(1)
        else:
            print(err)
            exit(1)

In [8]:
#create tables
#create tables from dictionary
def create_tables(tables, cursor):
    for table_name in tables:
        table_description = tables[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print('already exists')
            else:
                print(err.msg)
        else:
            print('OK')

In [9]:
#get business data from Yelp
#connects to yelp api and pulls businesses data

#return json object of data from Yelp
#for businesses
def businessCallGET(args):
    businessEndpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'bearer %s'% config.APIKey}
    return requests.get(businessEndpoint, params=args, headers=headers)

In [23]:
#cleaning the businesses data

def cleanData(dat, keys):
    #loops each business
    clean_set = []
    for i in range(len(dat['businesses'])):
        something = []
        #cleaning loop
        for x in keys:
            try :
                if (x == 'price'):
                    something.append(len(dat['businesses'][i][x]))
                elif (x == 'location'):
                    str_piece = ''
                    if (len(dat['businesses'][i][x]['display_address']) > 0):
                        str_piece = dat['businesses'][i][x]['display_address'][0]
                        if (len(dat['businesses'][i][x]['display_address']) > 1):
                            str_piece += dat['businesses'][i][x]['display_address'][1]
                    else:
                        str_piece = 'null'
                    something.append(str_piece)
                else:
                    something.append(dat['businesses'][i][x])
            except  KeyError:
                #if the key doesn't exist
                something.append('null')
            #we have a single list of one row of data
            
        #return tuple of cleaned data
        #used in cur.execute(columns, data1,data2,etc)
        clean_set.append( (something[0], something[1], something[2],
            something[3], something[4], something[5]) )
    return clean_set

In [24]:
#inserts data into the table with the add_business column headers
def insertData(cursor, column_names, cleaned_data):
        try:
            print('in try block to insert into database')
            cursor.executemany(column_names, cleaned_data)
            print('successful')
        except mysql.connector.Error as err:
            print(err)
            print('\n')
            print('\n')
            print('ERROR DID NOT INSERT')
            print('\n')
            print('\n')
        cnx.commit()

In [7]:
#getting review data
# Create a table for the reviews
def reviewCallGET(id_num):
    reviewEndpoint_base = 'https://api.yelp.com/v3/businesses/'
    reviewEndpoint_end = '/reviews'
    return requests.get(reviewEndpoint_base + id_num + reviewEndpoint_end,headers=headers)

test = 'ysqgdbSrezXgVwER2kQWKA'
# review_params={'id':'ysqgdbSrezXgVwER2kQWKA'}
review_data=reviewCallGET(test).json()
print(review_data)

NameError: name 'headers' is not defined

In [26]:
import pandas as pd
cur.execute("""
SELECT *
FROM PizzaPlaces.businesses
;
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,id,name,review_count,rating,price,address
0,04xHcYsqA3jCzpsX1Vs8ZQ,Randazzo's Pizzeria,42,3.50,1,"306 Ditmas AveBrooklyn, NY 11218"
1,0ROpM3UcZhprN6dhwK2Hsw,Joe & Pat's Pizzeria,160,4.00,2,"168 1st AveNew York, NY 10009"
2,1_LlUsiW1Pr01MEgv1dWvw,Rocky's & Nicky's Pizzeria,34,3.50,1,"6824 Colonial RdBrooklyn, NY 11220"
3,2Q6-R8l_ecJNPmmHA9iIkA,La Romana Pizzeria,16,4.50,1,"755 Grand StBrooklyn, NY 11211"
4,3rGroJSZlLgL81QsQKukCA,Bravi Ragazzi,104,3.50,2,"570 Putnam AveBrooklyn, NY 11221"
5,5AozkAQKOfRMEodf_FYfww,Sauce and Soda,26,4.00,0,"40 Wyckoff AveBrooklyn, NY 11237"
6,5aU2wWbj78R1k9Bzni8xNA,Madison's Pizzeria,1,5.00,0,"775 Madison StRidgewood, NY 11385"
7,6ECcDYsRgmfv0eKXk-c11Q,La Nonna Krispy Krust Pizza,39,4.50,2,"175 Kent AveBrooklyn, NY 11211"
8,71gTwIVOynPCNOFLEv-bQA,LBK Pizza Napoletana,62,4.50,2,"669 Union StBrooklyn, NY 11215"
9,73HeiHqXlj8bF1GXXBrn1g,Dia,78,4.50,0,"58 2nd AveNew York, NY 10003"


In [ ]:
#1.
# SELECT name
# FROM PizzaPlaces.businesses
# ORDER BY review_count DESC
#2


In [ ]:
cur.execute('''
DROP TABLE PizzaPlaces.businesses
;
''')
cnx.commit()